In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill

In [2]:
week1_Sheets = ["Week1Customer","Week1Monitoring","Week1Transaction"]
week2_Sheets = ["Week2Customer","Week2Monitoring","Week2Transaction"]
file_path = r"C:\Users\saran\OneDrive - sarokshllc.com\BSA_AML_TEST_DATA.xlsx"
week1_df = [pd.read_excel(file_path,sheet_name=sheet) for sheet in week1_Sheets]
week2_df = [pd.read_excel(file_path,sheet_name=sheet) for sheet in week2_Sheets]

week1_df = pd.concat(week1_df,ignore_index=True)
week2_df = pd.concat(week2_df,ignore_index=True)


In [3]:
pd.set_option('display.max_rows',40)
pd.set_option('display.max_columns',40)

In [ ]:
week1_df.head()
week2_df.head()

In [4]:
# merge both week data
merged_data = week1_df.merge(week2_df, on=['Title','DataSourceID'],suffixes=("_week1","_week2"), how="outer")

In [5]:
# Compare values and flag changes
columns_to_check = [col for col in week1_df.columns if col not in ["Date", "DataSourceID", "Title"]]

# Add a "Has_Changes" column
merged_data["Has_Changes"] = merged_data.apply(
    lambda row: any(
        row.get(f"{col}_week1") != row.get(f"{col}_week2")
        for col in columns_to_check
        if f"{col}_week1" in merged_data.columns and f"{col}_week2" in merged_data.columns       
    ),
    axis=1
)

In [6]:
# Export to Excel with Highlighting
wb = Workbook()
ws = wb.active

# Write headers
headers = list(merged_data.columns)
ws.append(headers)

# Apply cell formatting for changes
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# Write data and apply highlights
for index, row in merged_data.iterrows():
    for col_idx, col in enumerate(headers, start=1):
        # Write the cell value
        ws.cell(row=index + 2, column=col_idx, value=row[col])
        
        # Highlight changes (check only matching _week1 and _week2 columns)
        if col.endswith("_week1") and col.replace("_week1", "") in columns_to_check:
            base_col = col.replace("_week1", "")  # Get the base column name
            week2_col = f"{base_col}_week2"      # Corresponding _week2 column
            if week2_col in headers:             # Ensure _week2 column exists
                value_week1 = row[col]
                value_week2 = row.get(week2_col)
                
                # Check if values are different, accounting for nulls
                if pd.notna(value_week1) and pd.notna(value_week2) and value_week1 != value_week2:
                    # Highlight _week1 and _week2 cells if values differ
                    ws.cell(row=index + 2, column=col_idx).fill = yellow_fill
                    ws.cell(row=index + 2, column=headers.index(week2_col) + 1).fill = yellow_fill
                elif (pd.isna(value_week1) and pd.notna(value_week2)) or (pd.notna(value_week1) and pd.isna(value_week2)):
                    # Highlight if one is null and the other is not
                    ws.cell(row=index + 2, column=col_idx).fill = yellow_fill
                    ws.cell(row=index + 2, column=headers.index(week2_col) + 1).fill = yellow_fill

# Save the Excel file
wb.save("Merged_Data_Highlighted.xlsx")

print("Task completed! Output saved to 'Merged_Data_Highlighted.xlsx'")

Task completed! Output saved to 'Merged_Data_Highlighted.xlsx'
